## Modelagem e limpeza dos dados

In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"

dados = pd.read_csv(uri)

dados.head()

,Unnamed: 0,preco,vendido,idade_do_modelo,km_por_ano
0,0,30941.02,1,18,35085.22134
1,1,40557.96,1,20,12622.05362
2,2,89627.50,0,12,11440.79806
3,3,95276.14,0,3,43167.32682
4,4,117384.68,1,4,12770.11290


In [2]:
dados = dados.drop(columns=["Unnamed: 0"], axis=1)

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


### Alguns detalhes gerais sobre as informações

In [3]:
missing = dados.isna().sum()

missing

preco              0
vendido            0
idade_do_modelo    0
km_por_ano         0
dtype: int64

In [4]:
dados.shape

(10000, 4)

In [5]:
dados.vendido.value_counts(normalize=True)

1    0.58
0    0.42
Name: vendido, dtype: float64

## Maneira incerta de fazer a validação dos Modelos

In [6]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC

x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)

print("Treinaremos com {} e testaremos com {} elementos.".format(len(treino_x), len(teste_x)))

Treinaremos com 7500 e testaremos com 2500 elementos.


In [7]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier(strategy="stratified")
dummy_stratified.fit(treino_x, treino_y)

taxa_acerto = dummy_stratified.score(teste_x, teste_y) * 100

print("A taxa de acerto do dummy stratified foi {:.2f}%".format(taxa_acerto))

A taxa de acerto do dummy stratified foi 50.96%


In [8]:
from sklearn.tree import DecisionTreeClassifier

SEED = 158020
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=3)
model.fit(treino_x, treino_y)
lol = model.predict_proba(treino_x)

taxa_acerto = model.score(teste_x, teste_y) * 100

print("A taxa de acerto da árvore de decisão foi {:.2f}%".format(taxa_acerto))

A taxa de acerto da árvore de decisão foi 76.00%


In [9]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

SEED = 88
np.random.seed(SEED)

treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size=0.25, stratify=y)

model = DecisionTreeClassifier(max_depth=3)
model.fit(treino_x, treino_y)
lol = model.predict_proba(treino_x)

taxa_acerto = model.score(teste_x, teste_y) * 100

print("A taxa de acerto da árvore de decisão foi {:.2f}%".format(taxa_acerto))

A taxa de acerto da árvore de decisão foi 79.96%


## Validação Cruzada (Cross Validation)

In [10]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=3)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x, y, cv=5)

media = resultados['test_score'].mean()
desvio_padrao = resultados['test_score'].std()

# O intervalo deve ser de Média -+ 2x Desvio_Padrão
print("O intervalo de precisão do nosso algoritmo é: {:.2f}% a {:.2f}%".format(((media-2*desvio_padrao)*100), ((media+2*desvio_padrao)*100)))

O intervalo de precisão do nosso algoritmo é: 77.78% a 79.60%


### Aleatoriedade no Cross Validation

In [11]:
def imprime_resultados(resultados):
    media = resultados['test_score'].mean()
    desvio_padrao = resultados['test_score'].std()
    print("Precisão Média: {:.2f}%".format(media*100))
    print("O intervalo de precisão: {:.2f}% a {:.2f}%".format(((media-2*desvio_padrao)*100), ((media+2*desvio_padrao)*100)))

#### Sem Shuffle

In [12]:
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

# Gerador de validação
cv = KFold(n_splits=10)

model = DecisionTreeClassifier(max_depth=3)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x, y, cv=cv)

imprime_resultados(resultados)

Precisão Média: 78.67%
O intervalo de precisão: 76.58% a 80.76%


#### Com Shuffle

In [13]:
# Gerador de validação
cv = KFold(n_splits=10, shuffle=True)

model = DecisionTreeClassifier(max_depth=3)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x, y, cv=cv)

imprime_resultados(resultados)

Precisão Média: 78.67%
O intervalo de precisão: 76.36% a 80.98%


## Simulação de dados horríveis

Pode ser que haja um desbalanceamento dos dados selecionados

In [14]:
dados_azar = dados.sort_values("vendido", ascending=True)

x_azar = dados_azar[["preco", "idade_do_modelo", "km_por_ano"]]
y_azar = dados_azar["vendido"]

dados_azar

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704
...,...,...,...,...
5491,71910.43,1,9,25778.40812
1873,30456.53,1,6,15468.97608
1874,69342.41,1,11,16909.33538
5499,70520.39,1,16,19622.68262


In [15]:
SEED = 158020
np.random.seed(SEED)

# Gerador de validação
cv = KFold(n_splits=10)

model = DecisionTreeClassifier(max_depth=3)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x_azar, y_azar, cv=cv)

imprime_resultados(resultados)

Precisão Média: 78.67%
O intervalo de precisão: 74.13% a 83.21%


In [16]:
SEED = 158020
np.random.seed(SEED)

# Gerador de validação
cv = KFold(n_splits=10, shuffle=True)

model = DecisionTreeClassifier(max_depth=3)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x_azar, y_azar, cv=cv)

imprime_resultados(resultados)

Precisão Média: 78.66%
O intervalo de precisão: 76.14% a 81.18%


In [17]:
from sklearn.model_selection import StratifiedKFold

SEED = 158020
np.random.seed(SEED)

# Gerador de validação
cv = StratifiedKFold(n_splits=10, shuffle=True)

model = DecisionTreeClassifier(max_depth=3)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x_azar, y_azar, cv=cv)

imprime_resultados(resultados)

Precisão Média: 78.68%
O intervalo de precisão: 77.33% a 80.03%


## Dados agrupáveis

### Gerando dados aleatórios

In [18]:
np.random.seed(SEED)
dados['modelo_aleatorio'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,17
1,40557.96,1,20,12622.05362,20
2,89627.50,0,12,11440.79806,12
3,95276.14,0,3,43167.32682,2
4,117384.68,1,4,12770.11290,2


In [19]:
dados.modelo_aleatorio.unique()

array([17, 20, 12,  2, 10,  9, 18,  4, 16,  6,  5, 21, 15, 13, 11, 14,  8,
        3, 19,  7, 22,  0,  1, -1], dtype=int64)

### Adaptação para melhores valores de modelo de carro

In [20]:
dados.modelo_aleatorio = dados.modelo_aleatorio + abs(dados.modelo_aleatorio.min()) + 1

dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo_aleatorio
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,22
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,4
4,117384.68,1,4,12770.11290,4


In [21]:
dados.modelo_aleatorio.unique()

array([19, 22, 14,  4, 12, 11, 20,  6, 18,  8,  7, 23, 17, 15, 13, 16, 10,
        5, 21,  9, 24,  2,  3,  1], dtype=int64)

#### Preparação para prever um possível modelo de carro novo

In [22]:
from sklearn.model_selection import GroupKFold

SEED = 158020
np.random.seed(SEED)

# Gerador de validação
cv = GroupKFold(n_splits=10)

model = DecisionTreeClassifier(max_depth=3)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x_azar, y_azar, cv=cv, groups=dados.modelo_aleatorio)

imprime_resultados(resultados)

Precisão Média: 78.68%
O intervalo de precisão: 76.27% a 81.09%


## Cross Validation com StandardScaler

In [23]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 158020
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)

treino_x_scaler = scaler.transform(treino_x)
teste_x_scaler = scaler.transform(teste_x)

model = SVC()
model.fit(treino_x_scaler, treino_y)

taxa_acerto = model.score(teste_x_scaler, teste_y) * 100

print("A taxa de acerto do SVC foi {:.2f}%".format(taxa_acerto))

A taxa de acerto do SVC foi 77.56%


In [24]:
SEED = 158020
np.random.seed(SEED)

# Gerador de validação
cv = GroupKFold(n_splits=10)

model = SVC()

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x_azar, y_azar, cv=cv, groups=dados.modelo_aleatorio)

imprime_resultados(resultados)

Precisão Média: 77.24%
O intervalo de precisão: 73.71% a 80.77%


### Ideia errada de escalonamento e de treino

In [25]:
scaler = StandardScaler()
scaler.fit(x_azar)

x_azar_scaler = scaler.transform(x_azar)

In [28]:
SEED = 158020
np.random.seed(SEED)

# Gerador de validação
cv = GroupKFold(n_splits=10)

model = SVC()

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(model, x_azar_scaler, y_azar, cv=cv, groups=dados.modelo_aleatorio)

imprime_resultados(resultados)

Precisão Média: 76.65%
O intervalo de precisão: 73.60% a 79.69%


## Criação de uma Pipeline

### Ideia correta

In [30]:
from sklearn.pipeline import Pipeline

SEED = 158020
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

# Gerador de validação
cv = GroupKFold(n_splits=10)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(pipeline, x_azar_scaler, y_azar, cv=cv, groups=dados.modelo_aleatorio)

imprime_resultados(resultados)

Precisão Média: 76.67%
O intervalo de precisão: 73.68% a 79.65%


In [31]:
from sklearn.ensemble import RandomForestClassifier

SEED = 158020
np.random.seed(SEED)

scaler = StandardScaler()
modelo = RandomForestClassifier()

pipeline = Pipeline([('transformacao', scaler), ('estimador', modelo)])

# Gerador de validação
cv = GroupKFold(n_splits=10)

# Validação cruzada do model com os dados antes do split, separados em 5 partes
resultados = cross_validate(pipeline, x_azar_scaler, y_azar, cv=cv, groups=dados.modelo_aleatorio)

imprime_resultados(resultados)

Precisão Média: 75.14%
O intervalo de precisão: 72.08% a 78.20%
